Required imports

In [1]:
import numpy as np
from scipy import special
from sklearn.base import ClassifierMixin, BaseEstimator
from sklearn.datasets import make_classification

First let's implement loss function calculation.  
In this task we'll use loss known as Log-Loss.

$$
L(\textbf{w}, \textbf{X}, \textbf{y}) = \sum_{i=1}^{N}log(1 + exp(-y_i x_i^T \textbf{w})) + \lambda_1 \sum_{j=1}^M |w_j| + \lambda_2 \sum_{j=1}^M w_j^2
$$

Here,   
$\textbf{X}$ is a design matrix, $\textbf{X} \in \mathbb{R}^{NxM}$  
$\textbf{x_i}$ is ith training example, $\textbf{x_i} \in \mathbb{R}^M$  
$\textbf{w}$ is weights vector to be optimized, $\textbf{w} \in \mathbb{R}^M$  
$y_i$ is the class of ith training example, $y_i \in \{-1, 1\}$  

Therefore we can rewrite it in matrix form:  
$$
L(\textbf{w}, \textbf{X}, \textbf{y}) = log(1 + exp(-(\textbf{X} \textbf{w})^T \textbf{y})) + \lambda_1 \| \textbf{w} \|_1 + \lambda_2 \| \textbf{w} \|_2
$$  

Here,   
$\|\textbf{a}\|_1$ is L1 norm  
$\|\textbf{a}\|_2$ is L2 norm  

In [104]:
def lossf(w, X, y, l1, l2):
    """
    :param w: numpy.array of size (M,) dtype = np.float
    :param X: numpy.array of size (N, M), dtype = np.float
    :param y: numpy.array of size (N,), dtype = np.int
    :param l1: float, l1 regularizer coefficient
    :param l2: float, l2 regularizer coefficient 
    :return: float, value of loss function
    """
    lossf = np.log(1 + np.exp(-np.matmul(np.matmul(X, w).T, y))) + \
                   l1 * np.linalg.norm(w, ord=1) + \
                   l2 * np.square(np.linalg.norm(w, ord=2))
    return lossf

Next we should implement gradient calculation.  
Gradient is a vector containing partial derivatives of a function
$$
\nabla F (x, y, z) = (\frac{\partial F}{\partial x}, \frac{\partial F}{\partial y}, \frac{\partial F}{\partial z})^T
$$

Therefore, we need to compute partial derivatives for target function.
$$
\frac{\partial L(\textbf{w}, \textbf{X}, \textbf{y})}{\partial w_j} = -\frac{exp(-(\textbf{X} \textbf{w})^T \textbf{y})}{1 + exp(-(\textbf{X} \textbf{w})^T \textbf{y}))} * \pmb{x}^{jT} \textbf{y} + \lambda_1 sgn (w_j) + 2 \lambda_2 w_j
$$

Here,  
$sgn(x)$ is signum function  
$x^j$ is vector containing values of feature j  

Rewrite it into matrix form:  
$$
\frac{\partial L(\textbf{w}, \textbf{X}, \textbf{y})}{\partial \textbf{w}} = -\frac{exp(-(\textbf{X} \textbf{w})^T \textbf{y})}{1 + exp(-(\textbf{X} \textbf{w})^T \textbf{y}))} * \textbf{X}^T \textbf{y}  + \lambda_1 sgn (\textbf{w}) + 2 \lambda_2 \textbf{w} = (\frac{1}{1 + exp(-(\textbf{X} \textbf{w})^T \textbf{y}))} - 1) * \textbf{X}^T \textbf{y}  + \lambda_1 sgn (\textbf{w}) + 2 \lambda_2 \textbf{w}
$$

In [160]:
def gradf(w, X, y, l1, l2):
    """
    :param w: numpy.array of size (M,), dtype = np.float
    :param X: numpy.array of size (N, M), dtype = np.float
    :param y: numpy.array of size (N,), dtype = np.int
    :param l1: float, l1 regularizer coefficient 
    :param l2: float, l2 regularizer coefficient 
    :return: numpy.array of isze (M,), dtype = np.float, gradient vector d lossf / dw
    """
    p = np.matmul(np.matmul(X, w).T, y)
    gradw = (special.expit(p) - 1) * np.matmul(X.T, y) + l1 * np.sign(w) + 2 * l2 * w
    return gradw

Next we need to implement learning with gradient descent method.  
The main idea here is to iteratively move towards the solution.

$$
\textbf{w} (t = n+1) = \textbf{w} (t = n) - \eta \nabla(\textbf{w})
$$

Here,
$\eta$ is a learning rate, variable determining speed of fitting weights.  

Also we need a function to predict class of new examples.  
Clearly, dot product of $\textbf{x_i}$ and $\textbf{w}$ is unbounded, so we need to squeeze our predictions to the range $[0, 1]$  
The most common choice for that is the sigmoid function:  
$\sigma(x) = \frac{1}{1 + exp(-x)}$

In [183]:
# -*- coding: utf-8 -*-

# Используйте scipy.special для вычисления численно неустойчивых функций
# https://docs.scipy.org/doc/scipy/reference/special.html#module-scipy.special


class LR(ClassifierMixin, BaseEstimator):
    def __init__(self, lr=1, l1=1e-4, l2=1e-4, num_iter=1000, verbose=0):
        self.l1 = l1
        self.l2 = l2
        self.w = None
        self.lr = lr
        self.verbose = verbose
        self.num_iter = num_iter

    def fit(self, X, y):
        """
        Logistic regression training.
        self.w is to be fitted here.

        self.verbose == True means output of loss values for every iteration

        :param X: numpy.array of size (N, M), dtype = np.float
        :param y: numpy.array of size (N,), dtype = np.int
        :return: self
        """
        
        n, d = X.shape
        self.w = np.random.randn(d)
        self.w[self.w < -3] = -3
        self.w[self.w > 3] = 3

        for i in range(self.num_iter):
            w_new = self.w - self.lr * gradf(self.w, X, y, self.l1, self.l2)
                
            if self.verbose != 0:
                print "Iteration " + str(i) + ". Current loss is " + str(lossf(self.w, X, y, self.l1, self.l2))
            self.w = w_new
        return self

    def predict_proba(self, X):
        """
        Predict probability of belonging objects to class 1.

        :param X: numpy.array of size (N, M), dtype = np.float
        :return: numpy.array of size (N,), dtype = np.int
        """
        return special.expit(np.matmul(X, self.w))

    def predict(self, X):
        """
        Class prediction
        Returns np.array of size(N,) of 1 and -1.

        :param X: numpy.array of size (N, M), dtype = np.float
        :return:  numpy.array of size (N,), dtype = np.int
        """
        predicts = (self.predict_proba(X) > .5) * 2 - 1
        return predicts 


def test_work():
    X, y = make_classification(n_features=100, n_samples=1000)
    print "Start test"

    y = 2 * (y - 0.5)

    try:
        clf = LR(lr=1, l1=1e-4, l2=1e-4, num_iter=10000, verbose=0)
    except Exception:
        assert False, "Unable to create the model"
        return

    try:
        clf = clf.fit(X, y)
    except Exception:
        assert False, "Unable to fit the model"
        return

    print "Accuracy: " + str((clf.predict(X) == y).mean())

    assert isinstance(lossf(clf.w, X, y, 1e-3, 1e-3), float), "The loss function returns non-scalar value"
    assert gradf(clf.w, X, y, 1e-3, 1e-3).shape == (100,), "Wrong size of grad"
    assert gradf(clf.w, X, y, 1e-3, 1e-3).dtype == np.float, "Wrong type of grad"
    assert clf.predict(X).shape == (1000,), "Wrong size of prediction vector"
    assert np.min(clf.predict_proba(X)) >= 0, "Probabilities are less than 0"
    assert np.max(clf.predict_proba(X)) <= 1, "Probabilities are greater than 1"
    assert len(set(clf.predict(X))) == 2, "There are more than 2 classes!"
    print "End tests"

test_work()

Start test
Accuracy: 0.903
End tests
